In [ ]:
#1
#Imports
import pyarrow
import feather
import time
import json
import pandas
import csv
import nltk
import string
import re
import random
import matplotlib.pyplot as plt
import tables
import numpy as np
import pyarrow.feather as pf
import spacy
from spacy.matcher import Matcher
from datetime import datetime
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

from tables import *
from datetime import datetime


In [ ]:
#Read in data to testDF
path = 'all_april.feather'
aprilDF = feather.read_dataframe(path)



In [ ]:
#create sentiment array for april and may
sentimentValuesApril = list([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])


In [ ]:
#Calculates the average sentiment value for each day in April
tic = time.perf_counter()

#create list holding sentiment values
sentimentValuesApril = list([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

date = '-04-2020' #date stub for April 2020
for i in range(0,30): #for each day
    temp_date = date #reset the date variable
    if (i < 9): #for single digit days, prepend a zero to the date
        temp_date = '0' + str(i+1) + date
    else: 
        temp_date = str(i+1) + date;
        
    condition = aprilDF['date'] == temp_date #filter dataframe to a specific day
    for entry in aprilDF[condition]['sentiment']: #For each tweet for that day, add the sentiment to a running total
        sentimentValuesApril[i] += entry
    sentimentValuesApril[i] /= aprilDF[condition]['sentiment'].size #divide total by number of tweets
    
toc = time.perf_counter()
print(toc-tic)

In [ ]:
#Sets the variable allSentimentValues to our loaded in data 
allSentimentValues = sentimentValuesApril
print(len(allSentimentValues))

In [ ]:
#creates a list containing each date in april to be used for plotting our graphs
dateListApril = aprilDF['date'].unique().tolist()
i=0
for date in dateListApril:
    dateListApril[i] = str(date[0:5:1])
    i += 1;



totalDateList = dateListApril # + dateListMay

print(totalDateList)

In [ ]:
#load in covid figures #Look at files for how data was gathered
caseDF = pandas.read_csv('covidcases.csv', index_col=0)

unitedStates = list(caseDF['US'])
unitedStates = unitedStates[:44]
unitedKingdom = list(caseDF['UK'])
unitedKingdom = unitedKingdom[:44]
unitedAll = [sum(t) for t in zip(unitedKingdom, unitedStates)]
print(unitedStates)
print(unitedKingdom)
print(unitedAll)

In [ ]:
#Function to calculate the value of each value subtract the mean
def calcVSM(offset, N, x, y):
    xVSM = list()
    yVSM = list()
    productVSM = list()
    for val in x[offset:offset+N]:
        xVSM.append(val - np.mean(x[offset:offset+N]))
    for val in y[:N]:
        yVSM.append(val - np.mean(y[:N]))
    for i in range(N):
        productVSM.append(xVSM[i] * yVSM[i])
        
    return productVSM

In [ ]:
#Cross correlation equation
N = 30
x = unitedAll
y = allSentimentValues
productVSM = calcVSM(0, N, x, y)
r = (1/N) * ((sum(productVSM))/np.sqrt(np.var(x[:N]) * np.var(y[:N])))

print(r)

In [ ]:
#cross corerlation for each offset up to two weeks
N = 30 #number of days being looked at
x = unitedAll

y = allSentimentValues
corrArray = list()
for i in range(0,14,1):
    productVSM = calcVSM(i, 30, x, y)
    corrArray.append((1/(N)) * ((sum(productVSM))/np.sqrt(np.var(x[i:i+N]) * np.var(y[:N]))))
corrArray

In [ ]:
#Set varaible to our desired dataset
case_data = unitedAll

In [ ]:
#Fix the formatting of the dates 
subDates = list()
for date in totalDateList:
    subDates.append(date[:2])


In [ ]:
#Plot the combined US and UK against sentiment
sample_range = 30
x = subDates[:sample_range]
y = allSentimentValues[:sample_range]
fig, ax = plt.subplots()
ax.plot(x,y, 's-r', label = "Average Sentiment Score")

ax.set_ylim(ymin=0)
ax3 = ax.twinx()
ax3.plot(case_data[:sample_range], 'o-b', label = "Number of new cases")
ax3.set_ylim(ymin=0)
ax.xaxis.set_label_text("Date")
ax.yaxis.set_label_text("Average Sentiment Score")
ax3.yaxis.set_label_text("Number of cases")
lns = [ax, ax3]
ax.legend(loc="lower right")
ax3.legend(loc='lower right', bbox_to_anchor=(1.0, 0.1))
fig.subplots_adjust(right=2.0)

In [ ]:
#Plot the NCC ratio for a moving time lag
x = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
y = corrArray

fig, ax = plt.subplots()
ax.plot(x,y, 's-r', label="NCC")

ax.legend(loc="best")
ax.xaxis.set_label_text("Offset (days)")
ax.yaxis.set_label_text("Correlation ratio")



In [ ]:
#Graph for Correlation of covid keyword to cases


x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
y = corrArray2
#z = randomlist
#plt.figure(figsize=(15,5))
#plt.plot(x,y,z)
fig, ax = plt.subplots()
ax.plot(x,y, 's-r')

In [ ]:
#Graph showing the difference in correlation between 

x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
y = corrArray
fig, ax = plt.subplots()
ax.plot(x,y, 's-r', label = 'sentiment')
ax.legend(loc = 'center left')
ax3 = ax.twinx()

ax3.plot(corrArray2, 'o-b', label = "covid keyword")
ax3.legend(loc = 'upper left')
#fig.subplots_adjust(right=3.0)

In [ ]:
#Set up spacy matcher and choose pattern to test over dataframe
nlp = English()
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
#CHANGE THIS LINE TO CHANGE PATTERN
patterns = [nlp.make_doc(name) for name in ["fever","cough","taste","smell", "chills","breathe"]]
matcher.add("Names", patterns)

tic = time.perf_counter()

totalTestMatches_April = list([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
numMatchesApril = 0
date = '-04-2020'

for i in range(0,30):
    date2 = date
    if (i < 9):
        date2 = '0' + str(i+1) + date
    else:
        date2 = str(i+1) + date;
    totalMatches = 0
    condition = aprilDF['date'] == date2
    for row in aprilDF[condition]['text']:
        doc = nlp(row)
        if (len(matcher(doc)) > 0 ):
            totalMatches += 1
            numMatchesApril += 1
    totalTestMatches_April[i] = totalMatches / aprilDF[condition]['text'].size
        
        
toc = time.perf_counter()

print("aprilDF took: " + str(toc - tic) + " seconds" )
print("aprilDF matches: " + str(numMatchesApril))


In [ ]:
#calculate corr array for new pattern
N = 30 #(total number of days)

x = unitedAll
y = totalTestMatches_April
corrArrayFreq = list()
for i in range(0,14,1):
    productVSM = calcVSM(i, N, x, y)
    corrArrayFreq.append((1/(N)) * ((sum(productVSM))/np.sqrt(np.var(x[i:i+N]) * np.var(y[:N]))))
corrArrayFreq



In [ ]:
#Plot the combined US and UK against results for the keyword matching
sample_range = 30
x = subDates[:sample_range]
y = totalTestMatches_April[:sample_range]
fig, ax = plt.subplots()
ax.plot(x,y, 's-r', label = "Frequency")

ax.set_ylim(ymin=0)
ax3 = ax.twinx()
ax3.plot(case_data[:sample_range], 'o-b', label = "Number of new cases")
ax3.set_ylim(ymin=0)
ax.xaxis.set_label_text("Date")
ax.yaxis.set_label_text("% of tweets matching pattern")
ax3.yaxis.set_label_text("Number of cases")
lns = [ax, ax3]
ax.legend(loc="lower right")
ax3.legend(loc='lower right', bbox_to_anchor=(1.0, 0.1))
fig.subplots_adjust(right=2.0)

In [ ]:
#Plot the NCC for the data against a time lag
x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
y = corrArrayFreq

fig, ax = plt.subplots()
ax.plot(x,y, 's-r', label = "NCC")
ax.legend(loc="best")
ax.xaxis.set_label_text("Offset (days)")
ax.yaxis.set_label_text("Correlation ratio")

